In [1]:
import requests
import json

# Your Mapbox access token
access_token = 'pk.eyJ1IjoieW9obWFuIiwiYSI6IkxuRThfNFkifQ.u2xRJMiChx914U7mOZMiZw'

# API endpoint for the Mapbox Tilesets API
url = 'https://api.mapbox.com/v4/mapbox.mapbox-streets-v8/tilequery/{lon},{lat}.json?access_token={token}'

# Coordinates for the location (longitude, latitude)
lon, lat = -73.9857, 40.7484  # Example: near the Empire State Building

# Request URL
request_url = url.format(lon=lon, lat=lat, token=access_token)

# Make the request to Mapbox API
response = requests.get(request_url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract GeoJSON features
    features = data['features']
    
    # Create a GeoJSON structure
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    
    # Save to a GeoJSON file
    with open('buildings.geojson', 'w') as f:
        json.dump(geojson, f)
else:
    print(f"Error: {response.status_code}")


In [4]:
!pip install mapbox_vector_tile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 1.7 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 7.6 MB/s eta 0:00:00


In [20]:
import requests
import json
import mercantile
import mapbox_vector_tile
from shapely.geometry import shape, mapping
from shapely.ops import transform as shapely_transform
from functools import partial
import pyproj

# Your Mapbox access token
access_token = 'pk.eyJ1IjoieW9obWFuIiwiYSI6IkxuRThfNFkifQ.u2xRJMiChx914U7mOZMiZw'

# Define the center point (latitude, longitude)
center_lat = 35.86374571462002
center_lon = 140.01925645394718

# Define the distance in kilometers
distance_km = 5

# Approximate radius of the earth in kilometers
earth_radius_km = 6371.0

# Calculate the change in latitude and longitude
delta_lat = (distance_km / earth_radius_km) * (180 / math.pi)
delta_lon = (distance_km / (earth_radius_km * math.cos(math.pi * center_lat / 180))) * (180 / math.pi)

# Define the bounding box
min_lat = center_lat - delta_lat
max_lat = center_lat + delta_lat
min_lon = center_lon - delta_lon
max_lon = center_lon + delta_lon

# Get the tiles that intersect the bounding box at a specific zoom level
zoom_level = 15  # Zoom level for detailed building data
tiles = list(mercantile.tiles(min_lon, min_lat, max_lon, max_lat, zoom_level))

features = []

for tile in tiles:
    tile_url = f'https://api.mapbox.com/v4/mapbox.mapbox-streets-v8/{tile.z}/{tile.x}/{tile.y}.mvt?access_token={access_token}'
    
    response = requests.get(tile_url)
    if response.status_code == 200:
        tile_data = mapbox_vector_tile.decode(response.content)
        
        # Extract building features
        if 'building' in tile_data:
            ul = mercantile.ul(tile.x, tile.y, tile.z)
            # Width and height of the tile in geographic coordinates
            tile_width = mercantile.ul(tile.x + 1, tile.y, tile.z).lng - ul.lng
            tile_height = ul.lat - mercantile.ul(tile.x, tile.y + 1, tile.z).lat
            for feature in tile_data['building']['features']:
                geom = shape(feature['geometry'])
                # Transform tile coordinates to geographic coordinates
                transform = [
                    tile_width / 4096, 0, 0,
                    -tile_height / 4096, ul.lng, ul.lat
                ]
                geom = affine_transform(geom, transform)
                
                # Reproject to EPSG:4326
                project = partial(
                    pyproj.transform,
                    pyproj.Proj(init='epsg:3857'),  # Source coordinate system (Web Mercator)
                    pyproj.Proj(init='epsg:4326'))  # Destination coordinate system (WGS84)
                geom = shapely_transform(project, geom)
                
                geojson_feature = {
                    "type": "Feature",
                    "geometry": mapping(geom),
                    "properties": feature['properties']
                }
                features.append(geojson_feature)

# Create a GeoJSON structure
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save to a GeoJSON file
with open('buildings.geojson', 'w') as f:
    json.dump(geojson, f)

print("GeoJSON file 'buildings.geojson' created successfully.")


/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.

GeoJSON file 'buildings.geojson' created successfully.


/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Applications/miniconda3/envs/gis/lib/python3.11/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.

In [21]:
# map buildings.geojson
import folium

# Create a map centered at the location
m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add the GeoJSON data
folium.GeoJson('buildings.geojson').add_to(m)

# Display the map
m